In [1]:
# 主要ライブラリのimport
import pandas as pd
import numpy as np

#ファイル出力時にパスを指定する際によく使う
import os
import pathlib

# ailab_tools → ailabで作っている各種ツール群
# impala、GSheet、GMail等の操作が簡単にできる
# ドキュメントは下記
# https://s3-ap-northeast-1.amazonaws.com/ailab-smn-valis/ailab-tools/docs/html/index.html

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

#グラフをnotebook上で描画するためのおまじない
%matplotlib inline

INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
/opt/anaconda3/lib/python3.8/site-packages/ailab_tools/utils/pandas_utils.py:118: FutureWarning: The SparseDataFrame class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  sep: str = ",", binary: bool = True) -> pd.SparseDataFrame:


In [3]:
# Impala接続情報 hc5
ircfg = {
    "hosts": ['172.16.60.117'],
    "port": 21050,
    "user": 'vmspool',
    "request_pool": 'adhoc_dm01_pool',
    "httpfs_host": '172.16.60.42',
    "httpfs_port": 14000
}

# SSP共通案件に限定してSSP比較

In [4]:
# 抽出期間指定（前月1日～前月末日）
start_at = "2025-03-01"
end_at = "2025-03-31"

## A.0 共通案件の抽出

In [8]:
# 除外リスト（広告主✖️SSP✖️OS(is_app)でCTV1が1000を超えるものを対象）
exclude_ads_id = "23626,24377,8139,24580,15701,23336,16763,8426,5930"

In [9]:
query = (
f"""
select 
concat_ws('-', year, month) ym,
ads.child_ssp_id,
s.name ssp,
    ads.advertiser_id,
    advertiser_name,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'video' else 'display' end creative,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2) ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
left join console.ssp s on ads.child_ssp_id=s.id
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and ads.advertiser_id not in({exclude_ads_id})
and child_ssp_id>0
group by 1,2,3,4,5,6,7,8
"""
)
with ImpalaResource(**ircfg) as ir:
    data_a_0_1 = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [10]:
# path定義
parent_path = pathlib.Path("__file__").resolve().parent
file_path = parent_path / "data"/f"SSP比較共有案件確認_{start_at}_{end_at}_a_0_1.tsv"

data_a_0_1.to_csv(file_path, index=False, sep="\t")

## A1.SSP全体比較

In [ ]:
日付とAdvertiser_id更新で、A1,A4読み込む。

In [12]:
pc_display_advertiser_id="22747,24611,23425,8426,24624,23844,24027,19539,23278,18008,7694,46,22532,16777,22903,22693,22625,24786,16483,23098,22749,22437,7294,18937,7783,6931,24943,1737,20909,16754,19242,23906,3066,21875,22191,1373,24957,23704,9700,7151,22559,14398,23782,22607,14508,6229,867,18141,24245,19950,12766"
pc_native_advertiser_id="23278,11092,22747,24624,3066,16777,23146,16754,20909,24686,22749,23414,23338,5900,6229,24832,19552,22607,21871,20870,24921,22693,24957,8139,7294,24943,22532,19363,18600,14398,23282,23906,13194,24529,16006,23289,24108,15372,8348,24493,22559,24553,11213,24447,23132,4978,12766,8178,46"
spweb_display_advertiser_id="19539,10847,18008,17989,24553,12766,17278,18861,24624,6862,24757,23278,23202,24243,22747,24513,24982,4264,21797,13837,16483,21638,23696,23282,23194,23191,23155,24237,19679,24608,22532,7983,24027,4576,22693,18989,24979,24945,24523,23980,24711,17886,16737,24126,23704,24583,24717,23356,24832,24033,24611,24606,23813,6931,22191,23432,19818,24218,20810,16777,17800,17050,21986,22412,23844,24803,20870,23188,20886,24786,24192,21985,23414,10107,8139,24245,23098,24296,23695,22392,24943,19951,24487,22718,22558,22672,25070,19171,20974,24699,24911,23435,8479,23419,20909,21871,23429,23275,16754,16203,9700,24896,23171,24627,9662,23289,25050,18937,24217,53,19033,11703,24957,20785,12000,24912,24715,10215,24330,25007,1905,24812,18110,6358,17364,23878,6229,22316,24654,1782,23180,19052,3066,24069,19163,24253,19696,17753,18342,23711,24726,16683,23028,24598,19574,23726,24348,19103,22926,24732,23489,22187,24101,24768,15941,21367,23720,24166,8426,17424,24690,24022,22610,24267,24138,23628,22738,17746,22466,24708,22315,21971,24524,23338,24918,24316,25002,24946,19693,21964,22631,22437,8387,7151,24336,12345,23407,19397,11343,22394,7783,23277,23038,23828,24585,22079,24814,24581,7294,21668,23960,10919,11544,22553,8595,21601,21912,23691,23705,17243,24331,24359,24574,24652,23343,23770,23904,22069,23315,22912,14398,24792,23800,24315,19552,19532,17593,20970,22305,13194,24349,18837,24623,20905,12188,23873,2763,16843,20832,24294,24590,23111,15280,24363,24074,22470,16756,24350,24329,15750,24931,24193,21605,24447,7078,24769,18880,22951,19588,3655,23415,3629,24621,10156,9324,24898,15362,10632,13227,22524,16006,12597,19061,13195,24719,24980,23424,2663,13019,18792,23964,19950,21815,22441,24605,13104,23877,24108,17579,5938,14422,24939,16656,18233,24811,23981,16593,1210,24729,22887,23328,22970,24515,9676,7821,1021,22950,25025,24936,25016,18294,22739,24785,24852,23450,10648"
spweb_native_advertiser_id="18861,24624,22747,24938,17886,24728,24243,10107,12766,25007,23414,24459,22532,24982,4264,24513,22558,16777,15941,3066,24350,23903,19818,22672,24490,18294,17050,24957,21638,20909,18880,24945,20870,8139,5900,24769,12000,9700,14398"
spapp_display_advertiser_id="10847,18008,23194,6862,23278,24513,23282,22693,24245,24218,19539,22532,17050,24553,23202,24243,23191,24624,13837,24832,22519,20810,24768,23704,23800,12766,20870,24217,24786,18342,23432,24598,23813,25070,3066,17886,23844,24982,17364,24606,24583,53,8139,24359,18110,22558,23711,20974,24945,24896,16483,22394,22187,2235,20909,9700,23098,23498,17593,24605,23277,4428,19052,23878,7783,15941,20970,24627,11200,24581,4576,23696,6358,24033,18937,12345,24911,23828,22315,23182,25007,24166,18792,24876,24585,9662,24126,23236,6229,21588,22937,14032,24193,22895,13194,19693,7294,16843,16763,24515,22079,24348,16777,24524,20886,19951,24316,13019,23424,1210,22466,19964,24138,22441,17800,17832,14398,16656,4978,24854,24852,24946,25026,19552,19174,24918,14574,3655,23964,17312,5900,22393,23981,24811,23877,25071,24971,2763,13104,20968,13696,21539,12188,23808,24792,22749,12304,4996,18294,21577"
spapp_native_advertiser_id="10847,24982,24945,25007,24513,10107,18861,24553,24624,22693,22558,15941,17800,24606,22532,17050,23403,17886,24243,19818,12766,23414,24102,24583,16754,24627,23419,23278,24911,3066,18600,16877,8139,25037,23878,4428,16160,22610,24126"

In [13]:
# pc_display
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
	sum(logicad_request_id_1_count_synced) logicad_request_id_1_count_synced,
	sum(logicad_request_id_1_sum_native_plcmtcnt_synced) logicad_request_id_1_sum_native_plcmtcnt_synced,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when native_request=true then 'native' when video_request=true then 'video' else 'display' end creative,
	child_ssp_id,
sum(logicad_request_id_1_count_synced) logicad_request_id_1_count_synced,
sum(logicad_request_id_1_sum_native_plcmtcnt_synced)logicad_request_id_1_sum_native_plcmtcnt_synced,
	0 res,
	0 bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_req
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family not in('iOS','Android')
and is_app=false
and native_request=false and video_request=false
group by 1,2,3,4,5

union all

select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when native_request=true then 'native' when video_request=true then 'video' else 'display' end creative,
	child_ssp_id,
	0 logicad_request_id_1_count_synced,
	0 logicad_request_id_1_sum_native_plcmtcnt_synced,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res ads
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family not in('iOS','Android')
and is_app=false
and creative_type not in(5,6,10)
and ads.advertiser_id in ({pc_display_advertiser_id})
group by 1,2,3,4,5

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'video' else 'display' end creative,
	child_ssp_id,
	0 logicad_request_id_1_count_synced,
	0 logicad_request_id_1_sum_native_plcmtcnt_synced,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family not in('iOS','Android')
and is_app=false
and creative_type not in(5,6,10)
and ads.advertiser_id in ({pc_display_advertiser_id})
group by 1,2,3,4,5) a
left join console.ssp s on s.id=a.child_ssp_id
group by  1,2,3,4,5,6
"""
)
with ImpalaResource(**ircfg) as ir:
    data_1_pc_display = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [14]:
# pc_native
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
	sum(logicad_request_id_1_count_synced) logicad_request_id_1_count_synced,
	sum(logicad_request_id_1_sum_native_plcmtcnt_synced) logicad_request_id_1_sum_native_plcmtcnt_synced,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when native_request=true then 'native' when video_request=true then 'video' else 'display' end creative,
	child_ssp_id,
sum(logicad_request_id_1_count_synced) logicad_request_id_1_count_synced,
sum(logicad_request_id_1_sum_native_plcmtcnt_synced)logicad_request_id_1_sum_native_plcmtcnt_synced,
	0 res,
	0 bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_req
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family not in('iOS','Android')
and is_app=false
and native_request=true and video_request=false
group by 1,2,3,4,5

union all

select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when native_request=true then 'native' when video_request=true then 'video' else 'display' end creative,
	child_ssp_id,
	0 logicad_request_id_1_count_synced,
	0 logicad_request_id_1_sum_native_plcmtcnt_synced,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res ads
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family not in('iOS','Android')
and is_app=false
and creative_type in(5,6)
and ads.advertiser_id in ({pc_native_advertiser_id})
group by 1,2,3,4,5

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'video' else 'display' end creative,
	child_ssp_id,
	0 logicad_request_id_1_count_synced,
	0 logicad_request_id_1_sum_native_plcmtcnt_synced,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family not in('iOS','Android')
and is_app=false
and creative_type in(5,6)
and ads.advertiser_id in ({pc_native_advertiser_id})
group by 1,2,3,4,5) a
left join console.ssp s on s.id=a.child_ssp_id
group by  1,2,3,4,5,6
"""
)
with ImpalaResource(**ircfg) as ir:
    data_1_pc_native = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [15]:
# spweb_display
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
	sum(logicad_request_id_1_count_synced) logicad_request_id_1_count_synced,
	sum(logicad_request_id_1_sum_native_plcmtcnt_synced) logicad_request_id_1_sum_native_plcmtcnt_synced,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when native_request=true then 'native' when video_request=true then 'video' else 'display' end creative,
	child_ssp_id,
sum(logicad_request_id_1_count_synced) logicad_request_id_1_count_synced,
sum(logicad_request_id_1_sum_native_plcmtcnt_synced)logicad_request_id_1_sum_native_plcmtcnt_synced,
	0 res,
	0 bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_req
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=false
and native_request=false and video_request=false
group by 1,2,3,4,5

union all

select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when native_request=true then 'native' when video_request=true then 'video' else 'display' end creative,
	child_ssp_id,
	0 logicad_request_id_1_count_synced,
	0 logicad_request_id_1_sum_native_plcmtcnt_synced,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res ads
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=false
and creative_type not in(5,6,10)
and ads.advertiser_id in ({spweb_display_advertiser_id})
group by 1,2,3,4,5

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'video' else 'display' end creative,
	child_ssp_id,
	0 logicad_request_id_1_count_synced,
	0 logicad_request_id_1_sum_native_plcmtcnt_synced,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=false
and creative_type not in(5,6,10)
and ads.advertiser_id in ({spweb_display_advertiser_id})
group by 1,2,3,4,5) a
left join console.ssp s on s.id=a.child_ssp_id
group by  1,2,3,4,5,6
"""
)
with ImpalaResource(**ircfg) as ir:
    data_1_spweb_display = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [16]:
# spweb_native
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
	sum(logicad_request_id_1_count_synced) logicad_request_id_1_count_synced,
	sum(logicad_request_id_1_sum_native_plcmtcnt_synced) logicad_request_id_1_sum_native_plcmtcnt_synced,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when native_request=true then 'native' when video_request=true then 'video' else 'display' end creative,
	child_ssp_id,
sum(logicad_request_id_1_count_synced) logicad_request_id_1_count_synced,
sum(logicad_request_id_1_sum_native_plcmtcnt_synced)logicad_request_id_1_sum_native_plcmtcnt_synced,
	0 res,
	0 bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_req
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=false
and native_request=true and video_request=false
group by 1,2,3,4,5

union all

select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when native_request=true then 'native' when video_request=true then 'video' else 'display' end creative,
	child_ssp_id,
	0 logicad_request_id_1_count_synced,
	0 logicad_request_id_1_sum_native_plcmtcnt_synced,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res ads
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=false
and creative_type in(5,6)
and ads.advertiser_id in ({spweb_native_advertiser_id})
group by 1,2,3,4,5

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'video' else 'display' end creative,
	child_ssp_id,
	0 logicad_request_id_1_count_synced,
	0 logicad_request_id_1_sum_native_plcmtcnt_synced,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=false
and creative_type in(5,6)
and ads.advertiser_id in ({spweb_native_advertiser_id})
group by 1,2,3,4,5) a
left join console.ssp s on s.id=a.child_ssp_id
group by  1,2,3,4,5,6
"""
)
with ImpalaResource(**ircfg) as ir:
    data_1_spweb_native = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [17]:
# spapp_display
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
	sum(logicad_request_id_1_count_synced) logicad_request_id_1_count_synced,
	sum(logicad_request_id_1_sum_native_plcmtcnt_synced) logicad_request_id_1_sum_native_plcmtcnt_synced,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when native_request=true then 'native' when video_request=true then 'video' else 'display' end creative,
	child_ssp_id,
sum(logicad_request_id_1_count_synced) logicad_request_id_1_count_synced,
sum(logicad_request_id_1_sum_native_plcmtcnt_synced)logicad_request_id_1_sum_native_plcmtcnt_synced,
	0 res,
	0 bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_req
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=true
and native_request=false and video_request=false
group by 1,2,3,4,5

union all

select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when native_request=true then 'native' when video_request=true then 'video' else 'display' end creative,
	child_ssp_id,
	0 logicad_request_id_1_count_synced,
	0 logicad_request_id_1_sum_native_plcmtcnt_synced,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res ads
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=true
and creative_type not in(5,6,10)
and ads.advertiser_id in ({spapp_display_advertiser_id})
group by 1,2,3,4,5

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'video' else 'display' end creative,
	child_ssp_id,
	0 logicad_request_id_1_count_synced,
	0 logicad_request_id_1_sum_native_plcmtcnt_synced,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=true
and creative_type not in(5,6,10)
and ads.advertiser_id in ({spapp_display_advertiser_id})
group by 1,2,3,4,5) a
left join console.ssp s on s.id=a.child_ssp_id
group by  1,2,3,4,5,6
"""
)
with ImpalaResource(**ircfg) as ir:
    data_1_spapp_display = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [18]:
# spapp_native
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
	sum(logicad_request_id_1_count_synced) logicad_request_id_1_count_synced,
	sum(logicad_request_id_1_sum_native_plcmtcnt_synced) logicad_request_id_1_sum_native_plcmtcnt_synced,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when native_request=true then 'native' when video_request=true then 'video' else 'display' end creative,
	child_ssp_id,
sum(logicad_request_id_1_count_synced) logicad_request_id_1_count_synced,
sum(logicad_request_id_1_sum_native_plcmtcnt_synced)logicad_request_id_1_sum_native_plcmtcnt_synced,
	0 res,
	0 bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_req
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=true
and native_request=true and video_request=false
group by 1,2,3,4,5

union all

select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when native_request=true then 'native' when video_request=true then 'video' else 'display' end creative,
	child_ssp_id,
	0 logicad_request_id_1_count_synced,
	0 logicad_request_id_1_sum_native_plcmtcnt_synced,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res ads
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=true
and creative_type in(5,6)
and ads.advertiser_id in ({spapp_native_advertiser_id})
group by 1,2,3,4,5

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'video' else 'display' end creative,
	child_ssp_id,
	0 logicad_request_id_1_count_synced,
	0 logicad_request_id_1_sum_native_plcmtcnt_synced,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=true
and creative_type in(5,6)
and ads.advertiser_id in ({spapp_native_advertiser_id})
group by 1,2,3,4,5) a
left join console.ssp s on s.id=a.child_ssp_id
group by  1,2,3,4,5,6
"""
)
with ImpalaResource(**ircfg) as ir:
    data_1_spapp_native = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [19]:
data_a_1=pd.concat([data_1_pc_display,data_1_pc_native,data_1_spweb_display,data_1_spweb_native,data_1_spapp_display,data_1_spapp_native])

In [20]:
# path定義
parent_path = pathlib.Path("__file__").resolve().parent
file_path = parent_path / "data"/ f"SSP_limited_adv__{start_at}_{end_at}_a_1.tsv"

data_a_1.to_csv(file_path, index=False, sep="\t")

## A4.商材、産業カテゴリ別SSP比較

In [21]:
# pc_display
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
advertiser_id,
advertiser_name,
case
when hie.private_brand_id = 107 then "7:TVBridge"
when hie.ad_type =5 then '6:DOOH'
when hie.oem_id=300 then '5:CPX'
when summary_name = "RTG" then "1:RTG"
when summary_name = "DyC" then "2:DyC"
when summary_name = "類似" then "3:類似拡張"
when summary_name in ("ATA","カスタム(ブロード)","動画","カスタムATA") then "4:オーディエンス(動画含)"
else "9:その他" end as category,
sangyo_name,
sangyo_concrete_name,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(
select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when native_request=true then 'native' when video_request=true then 'video' else 'display' end creative,
	child_ssp_id,
    a.target_id,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res a
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family not in('iOS','Android')
and is_app=false
and creative_type not in(5,6,10)
and a.advertiser_id in ({pc_display_advertiser_id})
group by 1,2,3,4,5,6

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'video' else 'display' end creative,
	child_ssp_id,
ads.target_id,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family not in('iOS','Android')
and is_app=false
and creative_type not in(5,6,10)
and ads.advertiser_id in ({pc_display_advertiser_id})
group by 1,2,3,4,5,6) a
left join console.ssp s on s.id=a.child_ssp_id
inner join dm.product_target_master c on c.target_id=a.target_id
inner join dm.hierarchies hie on a.target_id = hie.target_id
left join dm.sangyo_id san on hie.product_ctgr_id = san.product_ctgr_id
group by  1,2,3,4,5,6,7,8,9,10,11
"""
)
with ImpalaResource(**ircfg) as ir:
    data_4_pc_display = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [22]:
# pc_native
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
advertiser_id,
advertiser_name,
case
when hie.private_brand_id = 107 then "7:TVBridge"
when hie.ad_type =5 then '6:DOOH'
when hie.oem_id=300 then '5:CPX'
when summary_name = "RTG" then "1:RTG"
when summary_name = "DyC" then "2:DyC"
when summary_name = "類似" then "3:類似拡張"
when summary_name in ("ATA","カスタム(ブロード)","動画","カスタムATA") then "4:オーディエンス(動画含)"
else "9:その他" end as category,
sangyo_name,
sangyo_concrete_name,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(
select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when native_request=true then 'native' when video_request=true then 'video' else 'display' end creative,
	child_ssp_id,
    ads.target_id,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res ads
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family not in('iOS','Android')
and is_app=false
and creative_type in(5,6)
and ads.advertiser_id in ({pc_native_advertiser_id})
group by 1,2,3,4,5,6

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'video' else 'display' end creative,
	child_ssp_id,
ads.target_id,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family not in('iOS','Android')
and is_app=false
and creative_type in(5,6)
and ads.advertiser_id in ({pc_native_advertiser_id})
group by 1,2,3,4,5,6) a
left join console.ssp s on s.id=a.child_ssp_id
inner join dm.product_target_master c on c.target_id=a.target_id
inner join dm.hierarchies hie on a.target_id = hie.target_id
left join dm.sangyo_id san on hie.product_ctgr_id = san.product_ctgr_id
group by  1,2,3,4,5,6,7,8,9,10,11
"""
)
with ImpalaResource(**ircfg) as ir:
    data_4_pc_native = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [23]:
# spweb_display
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
advertiser_id,
advertiser_name,
case
when hie.private_brand_id = 107 then "7:TVBridge"
when hie.ad_type =5 then '6:DOOH'
when hie.oem_id=300 then '5:CPX'
when summary_name = "RTG" then "1:RTG"
when summary_name = "DyC" then "2:DyC"
when summary_name = "類似" then "3:類似拡張"
when summary_name in ("ATA","カスタム(ブロード)","動画","カスタムATA") then "4:オーディエンス(動画含)"
else "9:その他" end as category,
sangyo_name,
sangyo_concrete_name,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(
select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when native_request=true then 'native' when video_request=true then 'video' else 'display' end creative,
	child_ssp_id,
    ads.target_id,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res ads
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=false
and creative_type not in(5,6,10)
and ads.advertiser_id in ({spweb_display_advertiser_id})
group by 1,2,3,4,5,6

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'video' else 'display' end creative,
	child_ssp_id,
ads.target_id,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=false
and creative_type not in(5,6,10)
and ads.advertiser_id in ({spweb_display_advertiser_id})
group by 1,2,3,4,5,6) a
left join console.ssp s on s.id=a.child_ssp_id
inner join dm.product_target_master c on c.target_id=a.target_id
inner join dm.hierarchies hie on a.target_id = hie.target_id
left join dm.sangyo_id san on hie.product_ctgr_id = san.product_ctgr_id
group by  1,2,3,4,5,6,7,8,9,10,11
"""
)
with ImpalaResource(**ircfg) as ir:
    data_4_spweb_display = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [24]:
# spweb_native
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
advertiser_id,
advertiser_name,
case
when hie.private_brand_id = 107 then "7:TVBridge"
when hie.ad_type =5 then '6:DOOH'
when hie.oem_id=300 then '5:CPX'
when summary_name = "RTG" then "1:RTG"
when summary_name = "DyC" then "2:DyC"
when summary_name = "類似" then "3:類似拡張"
when summary_name in ("ATA","カスタム(ブロード)","動画","カスタムATA") then "4:オーディエンス(動画含)"
else "9:その他" end as category,
sangyo_name,
sangyo_concrete_name,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(
select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when native_request=true then 'native' when video_request=true then 'video' else 'display' end creative,
	child_ssp_id,
    ads.target_id,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res ads
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=false
and creative_type in(5,6)
and ads.advertiser_id in ({spweb_native_advertiser_id})
group by 1,2,3,4,5,6

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'video' else 'display' end creative,
	child_ssp_id,
ads.target_id,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=false
and creative_type in(5,6)
and ads.advertiser_id in ({spweb_native_advertiser_id})
group by 1,2,3,4,5,6) a
left join console.ssp s on s.id=a.child_ssp_id
inner join dm.product_target_master c on c.target_id=a.target_id
inner join dm.hierarchies hie on a.target_id = hie.target_id
left join dm.sangyo_id san on hie.product_ctgr_id = san.product_ctgr_id
group by  1,2,3,4,5,6,7,8,9,10,11
"""
)
with ImpalaResource(**ircfg) as ir:
    data_4_spweb_native = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [25]:
# spapp_display
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
advertiser_id,
advertiser_name,
case
when hie.private_brand_id = 107 then "7:TVBridge"
when hie.ad_type =5 then '6:DOOH'
when hie.oem_id=300 then '5:CPX'
when summary_name = "RTG" then "1:RTG"
when summary_name = "DyC" then "2:DyC"
when summary_name = "類似" then "3:類似拡張"
when summary_name in ("ATA","カスタム(ブロード)","動画","カスタムATA") then "4:オーディエンス(動画含)"
else "9:その他" end as category,
sangyo_name,
sangyo_concrete_name,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(
select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when native_request=true then 'native' when video_request=true then 'video' else 'display' end creative,
	child_ssp_id,
    ads.target_id,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res ads
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=true
and creative_type not in(5,6,10)
and ads.advertiser_id in ({spapp_display_advertiser_id})
group by 1,2,3,4,5,6

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'video' else 'display' end creative,
	child_ssp_id,
ads.target_id,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=true
and creative_type not in(5,6,10)
and ads.advertiser_id in ({spapp_display_advertiser_id})
group by 1,2,3,4,5,6) a
left join console.ssp s on s.id=a.child_ssp_id
inner join dm.product_target_master c on c.target_id=a.target_id
inner join dm.hierarchies hie on a.target_id = hie.target_id
left join dm.sangyo_id san on hie.product_ctgr_id = san.product_ctgr_id
group by  1,2,3,4,5,6,7,8,9,10,11
"""
)
with ImpalaResource(**ircfg) as ir:
    data_4_spapp_display = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [26]:
# spapp_native
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
advertiser_id,
advertiser_name,
case
when hie.private_brand_id = 107 then "7:TVBridge"
when hie.ad_type =5 then '6:DOOH'
when hie.oem_id=300 then '5:CPX'
when summary_name = "RTG" then "1:RTG"
when summary_name = "DyC" then "2:DyC"
when summary_name = "類似" then "3:類似拡張"
when summary_name in ("ATA","カスタム(ブロード)","動画","カスタムATA") then "4:オーディエンス(動画含)"
else "9:その他" end as category,
sangyo_name,
sangyo_concrete_name,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(
select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when native_request=true then 'native' when video_request=true then 'video' else 'display' end creative,
	child_ssp_id,
    ads.target_id,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res ads
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=true
and creative_type in(5,6)
and ads.advertiser_id in ({spapp_native_advertiser_id})
group by 1,2,3,4,5,6

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'video' else 'display' end creative,
	child_ssp_id,
ads.target_id,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=true
and creative_type in(5,6)
and ads.advertiser_id in ({spapp_native_advertiser_id})
group by 1,2,3,4,5,6) a
left join console.ssp s on s.id=a.child_ssp_id
inner join dm.product_target_master c on c.target_id=a.target_id
inner join dm.hierarchies hie on a.target_id = hie.target_id
left join dm.sangyo_id san on hie.product_ctgr_id = san.product_ctgr_id
group by  1,2,3,4,5,6,7,8,9,10,11
"""
)
with ImpalaResource(**ircfg) as ir:
    data_4_spapp_native = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [27]:
data_a_4=pd.concat([data_4_pc_display,data_4_pc_native,data_4_spweb_display,data_4_spweb_native,data_4_spapp_display,data_4_spapp_native])

In [28]:
# path定義
parent_path = pathlib.Path("__file__").resolve().parent
file_path = parent_path / "data"/ f"SSP_limited_adv__{start_at}_{end_at}_a_4_adv.tsv"

data_a_4.to_csv(file_path, index=False, sep="\t")